# artsy

In [1]:
import json

from geopy.geocoders import Nominatim
from pymongo import MongoClient

## setup

### database

In [2]:
client = MongoClient()
# client.drop_database('artsy')
db = client.artsy

In [5]:
for x in db.artists.find():
    print(x)
    break

{'updated_at': '2014-01-03T21:20:44+00:00', 'location': '', 'sortable_name': '03B Muren', 'birthday': '', 'hometown': '', '_links': {'artworks': {'href': 'https://api.artsy.net/api/artworks?artist_id=52c721e5b202a3edf1000072'}, 'genes': {'href': 'https://api.artsy.net/api/genes?artist_id=52c721e5b202a3edf1000072'}, 'permalink': {'href': 'http://www.artsy.net/artist/muren-03b'}, 'self': {'href': 'https://api.artsy.net/api/artists/52c721e5b202a3edf1000072'}, 'published_artworks': {'href': 'https://api.artsy.net/api/artworks?artist_id=52c721e5b202a3edf1000072&published=true'}, 'similar_artists': {'href': 'https://api.artsy.net/api/artists?similar_to_artist_id=52c721e5b202a3edf1000072'}, 'similar_contemporary_artists': {'href': 'https://api.artsy.net/api/artists?similar_to_artist_id=52c721e5b202a3edf1000072&similarity_type=contemporary'}}, 'nationality': '', 'gender': '', 'name': 'Muren 03B', '_id': '52c721e5b202a3edf1000072', 'created_at': '2014-01-03T20:47:33+00:00'}


In [7]:
x._id

AttributeError: 'dict' object has no attribute '_id'

### geolocator

In [26]:
geolocator = Nominatim()

In [32]:
geolocator.geocode('Seville').latitude

37.3886303

## Data

In [40]:
data = []
count = 0
for artist in db.artists.find():
    location = artist['location'] or artist['hometown']
    if location:
                artist_id = artist['_id']
                artists_artworks = db.artists_artworks.find_one({'_id': artist_id})
                if artists_artworks:
                    artwork_ids = artists_artworks['data']
                    for artwork_id in artwork_ids:
                        artworks_genes = db.artworks_genes.find_one({'_id': artwork_id})
                        count += 1
count

9497

In [ ]:
%%time
db.geolocation.drop()
total_count = db.artists.count()
size = 1000
chunks = [db.artists.find()[x:x+size] for x in range(0, total_count, size)]
for chunk in chunks:
    data = []
    for artist in chunk:
        location = artist['location'] or artist['hometown']
        if location:
            geo = geolocator.geocode(location)
            if geo:
                artist_id = artist['_id']
                artists_artworks = db.artists_artworks.find_one({'_id': artist_id})
                if artists_artworks:
                    artwork_ids = artists_artworks['data']
                    for artwork_id in artwork_ids:
                        artworks_genes = db.artworks_genes.find_one({'_id': artwork_id})
                        genes = []
                        if artworks_genes:
                            gene_ids = artworks_genes['data']
                            for gene_id in gene_ids:
                                gene = db.genes.find_one({'_id': gene_id})
                                if gene:
                                    genes.append(gene['name'])
                        info = {
                            '_id': artwork_id,
                            'latitude': geo.latitude,
                            'longitude': geo.longitude,
                            'genes': genes,
                        }
                        data.append(info)
    db.geolocation.insert_many(data)

In [ ]:
artworks = set()
genes = set()
relationships = set()
for geo in db.geolocation.find():
    artwork_node = 